### Assistants
1. Assistants 생성
2. Thread 생성
3. Thread 에 메세지 추가 - 사용자 질문 추가
4. Run

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import warnings

load_dotenv()
warnings.filterwarnings("ignore")

client = OpenAI()
Model = "gpt-4o-mini"

- 대화내용 기억

In [6]:
memory_buffer = [{"role": "developer", "content": "You are a helpful assistant"}]

while True:
    user_input = input("질문해 주세요")
    if user_input == "q":
        break

    # 사용자 입력을 메모리 버퍼에 추가
    memory_buffer.append({"role":"user", "content":user_input})

    response = client.responses.create(
        model=Model,
        input=memory_buffer,
        temperature=0.9,
        max_output_tokens=300
    )

    print(response.output_text)

어떤 종류의 음식을 원하시나요? 한식, 중식, 양식, 혹은 다른 나라 음식 중 어떤 게 좋으신가요? 또는 식사 스타일(가벼운, 푸짐한)도 말씀해 주시면 더 정확한 추천을 드릴 수 있어요!
물론입니다! 한식과 중식 위주로 몇 가지 점심 메뉴를 추천해드릴게요.

### 한식
1. **비빔밥** - 다양한 채소와 고기, 계란을 비벼 먹는 건강한 한 끼.
2. **김치찌개** - 따뜻하고 매콤한 김치찌개에 밥을 곁들여 먹으면 좋습니다.
3. **불고기** - 양념된 소고기를 볶아 밥과 함께 즐기기 좋습니다.
4. **갈비찜** - 부드러운 갈비에 다양한 채소를 넣고 찜한 요리.
5. **잡채** - 당면과 각종 채소, 고기를 볶아 만든 요리로, 맛과 식감이 뛰어납니다.

### 중식
1. **짜장면** - 달콤한 춘장 소스와 면이 잘 어우러진 인기 메뉴.
2. **짬뽕** - 매콤하고 시원한 국물의 해물 짬뽕.
3. **볶음밥** - 각종 재료를 넣고 볶아낸 맛있는 볶음밥.
4. **탕수육** - 바삭한 튀김옷에 달콤한 소스를 곁들인 인기 메뉴


Assistants API 사용

In [7]:
from typing_extensions import override
from openai import AssistantEventHandler

# Assistant 생성

assistant = client.beta.assistants.create(
    name="Fruit Advisor",
    instructions="너는 과일 지식이 풍부한 과일 전문가야. 나의 질문에 친절히 답변을 해주는 ChatBot 이야",
    model=Model
)

# Thread 생성
thread = client.beta.threads.create()

# 텍스트
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text):
        """
        텍스트 생성 완료 시점에 호출되는 함수
        """

        print("\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """
        텍스트 생성 중간에 호출되는 함수
        """

        print(delta.value, end="", flush=True)

# Run
# stream : 실시간으로 주고 받겠다는 의미
# streaming : 실시간 (현재 gpt 가 해주는 방식) / non-streaming : 전체 응답이 생성된 후 한번에 전달하는 개념
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="사용자를 고객님이라고 부르세요. 사용자에게 프리미엄 계정이 있습니다.",
    event_handler=EventHandler()
) as stream:
    stream.until_done()


assistant > 안녕하세요, 고객님! 무엇을 도와드릴까요? 필요한 정보나 질문이 있으시면 언제든지 말씀해 주세요.

In [8]:
while True:
    user_input = input("궁금한 과일에 대해 질문하세요")
    if user_input == "q":
        break

    # 사용자 입력 thread 에 추가
    response = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # 모델 쪽
    with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
    ) as stream:
        stream.until_done()


assistant > 수박은 여름철에 특히 인기가 많은 과일로, 수분이 많고 시원한 맛이 특징입니다. 수박은 주로 다음과 같은 특징을 가지고 있습니다:

1. **영양 성분**: 수박은 대부분이 수분(약 90%)으로 구성되어 있으며, 비타민 C와 비타민 A, 그리고 일부 항산화 물질인 라이코펜이 풍부하게 포함되어 있습니다.

2. **종류**: 수박은 다양한 품종이 있으며, 대표적으로는 씨가 있는 일반 수박과 씨 없는 수박이 있습니다. 또한, 작은 크기의 미니 수박도 인기가 있습니다.

3. **보관 방법**: 수박은 통째로 보관할 때 서늘한 곳에 두는 것이 좋으며, 한 번 자른 수박은 냉장고에 보관하시고 빠른 시일 내에 소비하는 것이 좋습니다.

4. **섭취 방법**: 수박은 그냥 생으로 먹는 것이 가장 일반적이지만, 주스, 스무디, 샐러드 등 다양하게 활용될 수 있습니다.

더 궁금한 점이나 특정한 질문이 있으시면 말씀해 주세요!
assistant > 바나나는 세계에서 가장 인기 있는 과일 중 하나로, 여러 가지 장점이 있는 건강식품입니다. 아래는 바나나에 대한 주요 정보입니다:

1. **영양 성분**: 바나나는 풍부한 탄수화물(주로 자연당), 식이섬유, 비타민 C, 비타민 B6, 칼륨, 마그네슘 등을 포함하고 있습니다. 특히 칼륨이 풍부하여 심혈관 건강에 도움이 됩니다.

2. **소화 도움**: 바나나에 포함된 식이섬유는 소화를 도와주며, 장 건강을 유지하는 데 기여합니다. 특히 바나나는 소화가 잘 되기 때문에 속이 불편할 때에도 좋습니다.

3. **에너지 공급**: 바나나는 빠르게 에너지를 공급해 주는 과일로, 운동 전후의 간식으로 적합합니다.

4. **보관 방법**: 바나나는 실온에서 보관이 가능하며, 잘 익은 바나나는 냉장고에서 보관할 수 있지만 껍질이 검게 변할 수 있습니다.

5. **섭취 방법**: 바나나는 그대로 먹는 것이 가장 일반적이며, 스무디, 요거트, 오트밀, 빵과 같은 다양한 요리에 추가할 수 있습니다. 또한, 구워 먹거나

### 문서 / 텍스트를 기반으로 챗봇 답변하기

In [ ]:
dataset="""Document content:
질문: UNDP의 eRecruit 시스템은 무엇입니까? 답변: UNDP의 eRecruit 시스템은 지원자가 하나 이상의 광고된 UNDP 채용 공고에 지원서를 제출하기 위해 정기적으로 업데이트할 수 있는 개인 프로필을 만들 수 있는 온라인 시스템입니다.
질문: UNDP의 eRecruit 시스템에 어떻게 액세스합니까? 답변: UNDP의 eRecruit 시스템은 다음 링크를 통해 접속할 수 있습니다: https://undpcareers.partneragency.org/erecruit.html
질문: UNDP의 eRecruit 시스템을 사용하여 지원하는 절차는 무엇입니까? 답변: 지원 절차 지원 절차
질문: UNDP의 eRecruit 시스템을 사용하여 온라인으로 지원해야 합니까? 답변: 모든 지원서는 UNDP의 eRecruit 시스템을 사용하여 온라인으로 제출해야 합니다. 오프라인 서면 지원서나 이메일을 통한 지원서는 접수되지 않습니다.
질문: UNDP의 eRecruit 시스템과 호환되는 브라우저는 무엇입니까? 답변: UNDP의 eRecruit 시스템은 Google Chrome, Internet Explorer 6 이상에 최적화되어 있습니다. 호환성 보기 모드를 사용하려면 Internet Explorer 9을 사용해야 합니다. 신청서가 성공적으로 제출되었는지 확인하려면 다음 브라우저 중 하나를 사용하는 것이 좋습니다.
질문: 내 프로필에 로그인하는 데 문제가 있습니다. 브라우저가 응답하지 않습니다. 어떻게 해야 하나요? 답변: 구인 신청을 위해 UNDP eRecruit 프로필에 로그인하는 데 문제가 있는 경우 이는 여러 가지 이유에서 비롯될 수 있으며 그 중 일부는 인터넷 연결과 같은 UNDP의 통제 범위를 벗어납니다. 그러나 이러한 유형의 문제를 해결하려면 다음 지침을 따르는 것이 좋습니다. • 권장 브라우저(및 버전)를 사용하고 있는지 확인하십시오. • 캐시/브라우저 기록을 지웁니다. http://www.refreshyourcache.com/en/home에서 브라우저 기록을 지우는 방법에 대한 정보를 찾을 수 있습니다. 이 작업을 수행하기 전에 브라우저에서 캐시를 지울 때의 결과를 알고 있는지 확인하십시오.
질문: UNDP의 eRecruit 시스템을 사용하는 데 도움이 필요하면 어떻게 합니까? 답변: UNDP의 eRecruit 시스템 사용에 대한 일반적인 질문이나 지원이 필요한 경우 헬프데스크(https://info.undp.org/sas/erecruit/Assets/HelpDesk.aspx)에 문의하세요.
질문: 왜 등록해야 합니까? 답변: 모든 지원자는 먼저 UNDP의 eRecruit 시스템에 등록해야 합니다. 등록이 완료되면 개인 정보를 입력하고 광고된 채용 공고에 지원할 수 있도록 개인 계정이 생성됩니다.
질문: 등록할 때 사용자 이름으로 무엇을 사용해야 합니까? 답변: UNDP eRecruit 시스템에 등록할 때 유효한 이메일 주소를 사용자 이름으로 사용하는 것이 좋습니다.
질문: 어떤 비밀번호 형식이 허용되나요? 답변: UNDP는 강력한 비밀번호 사용을 권장합니다. 비밀번호는 최소 8자 이상이어야 하며 문자와 숫자를 조합해야 합니다.
질문: 비밀번호를 어떻게 변경할 수 있나요? 답변: 시스템에 로그인한 후 '개인 정보' 링크에서 '비밀번호 변경' 옵션을 선택하세요. '사용자 이름 또는 비밀번호를 잊으셨나요?' 링크를 클릭하고 지침에 따라 잊어버린 비밀번호를 검색하세요.
질문: 비밀번호를 잊어버렸습니다. 어떻게 해야 하나요? 답변: 비밀번호를 잊어버린 경우, 사용자 ID 또는 비밀번호 찾기 링크를 클릭하고 두 가지 옵션 중 하나를 완료하세요.
질문: 내 데이터는 안전합니까? 답변: UNDP의 eRecruit 데이터는 개인 보안 데이터베이스에 저장되며 UNDP는 데이터의 소유자입니다. 이 시스템은 UNDP의 엄격한 보안 요구 사항을 충족합니다.
질문: 각 섹션의 모든 정보를 작성해야 합니까? 답변: 별표(*)가 표시된 모든 항목은 필수 정보이며 각 섹션별로 작성해야 합니다.
질문: 한 세션에서 모든 정보를 완료해야 합니까? 답변: 지원 과정 중 언제든지 지원서를 저장하고 나중에 계속할 수 있습니다. 섹션을 완료할 때 정보가 손실되지 않도록 정기적으로 저장 버튼을 사용하는 것이 좋습니다.
질문: 달력 기능을 사용하여 날짜를 어떻게 선택합니까? 답변: 날짜는 두 가지 방법으로 선택할 수 있습니다. A) dd/mm/yyyy 형식을 사용하여 필드에 날짜를 직접 입력하거나 B) 달력 아이콘을 클릭하여 날짜를 선택하고 연도, 월 및 일을 선택할 수 있습니다.
질문: 관련 정보가 드롭다운 옵션에 포함되어 있지 않으면 어떻게 해야 합니까? 답변: 사용 가능한 드롭다운 옵션 중에서 선택해야 합니다. 귀하의 개인 정보, 기술 및 경험에 가장 가까운 옵션을 선택하십시오.
질문: 조회 기능을 어떻게 사용합니까? 답변: 조회 기능을 사용하려면 돋보기 아이콘을 클릭하세요. 그런 다음 해당 필드에 전체 또는 부분 값을 입력하고 조회 버튼을 클릭합니다. 마지막으로 검색 결과에서 적절한 값을 선택하세요.
질문: 맞춤법 검사 기능을 사용할 수 있나요? 답변: 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 클릭 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 사전을 클릭하세요.
"""

